In [72]:
#导入相关的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
import missingno as msno
from scipy.sparse import csr_matrix
from sklearn.model_selection import KFold, StratifiedKFold
%matplotlib inline

In [73]:
#导入训练集
train = pd.read_csv('train_set.csv')
train.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [74]:
#导入测试集
test = pd.read_csv('test_set.csv')
test.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,25318,51,housemaid,married,unknown,no,174,no,no,telephone,29,jul,308,3,-1,0,unknown
1,25319,32,management,married,tertiary,no,6059,yes,no,cellular,20,nov,110,2,-1,0,unknown
2,25320,60,retired,married,primary,no,0,no,no,telephone,30,jul,130,3,-1,0,unknown
3,25321,32,student,single,tertiary,no,64,no,no,cellular,30,jun,598,4,105,5,failure
4,25322,41,housemaid,married,secondary,no,0,yes,yes,cellular,15,jul,368,4,-1,0,unknown


In [75]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse

test['y']=-1

cate_features = ['job', 'marital','education','default','housing','loan','contact','month','poutcome']
num_features = ['age', 'balance','day','duration','campaign','pdays','previous']

feature1 = cate_features + num_features
data = pd.concat([train,test])
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [76]:
# 特征工程1

def feat_count(df, df_feature,feature,value,name=""):
    df_count = pd.DataFrame(df_feature.groupby(feature)[value].count()).reset_index()
    if not name:
        df_count.columns = feature + [value+"_%s_count" % ("_".join(feature[0]))] 
    else:
        df_count.columns = feature + [name] # 注意这里表示df_count增加一列，且列名为[name]，上一行代码同理
    df = df.merge(df_count, on=feature, how="left")#.fillna(0)
    return df,df_count.columns[1]

for fe in feature1:
    data = feat_count(data,data,['ID'],fe)[0] # feat_count函数返回2个值，【0】表示第一个值

data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,...,contact_I_D_count,month_I_D_count,poutcome_I_D_count,age_I_D_count,balance_I_D_count,day_I_D_count,duration_I_D_count,campaign_I_D_count,pdays_I_D_count,previous_I_D_count
0,1,43,management,married,tertiary,no,291,yes,no,unknown,...,1,1,1,1,1,1,1,1,1,1
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,...,1,1,1,1,1,1,1,1,1,1
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,...,1,1,1,1,1,1,1,1,1,1
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,...,1,1,1,1,1,1,1,1,1,1
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,...,1,1,1,1,1,1,1,1,1,1


In [77]:
# 特征工程2

def feature_count(data, features):
    feature_name = 'count'
    for i in features:
        feature_name += '_' + i
    temp = data.groupby(features).size().reset_index().rename(columns={0: feature_name})
    data = data.merge(temp, 'left', on=features)
    return data,feature_name


ll=[]
for f in['campaign', 'contact','default','education','housing','job','loan','marital','poutcome','pdays','previous']:
    data,_=feature_count(data,['day','month',f])
    ll.append(_)

In [78]:
# 特征工程3

sparse_feature= ['contact','default','education','housing','job','loan','marital','month','poutcome']
dense_feature=['age', 'balance','day','duration','campaign','pdays','previous']

def get_new_columns(name,aggs):
    l=[]
    for k in aggs.keys():
        for agg in aggs[k]:
            if str(type(agg))=="<class 'function'>":
                l.append(name + '_' + k + '_' + 'other')
            else:
                l.append(name + '_' + k + '_' + agg)
    return l
for d in tqdm(sparse_feature):
    aggs={}
    for s in sparse_feature:
        aggs[s]=['count','nunique']
    for den in dense_feature:
        aggs[den]=['mean','max','min','std']
    aggs.pop(d)
    temp=data.groupby(d).agg(aggs).reset_index()
    temp.columns=[d]+get_new_columns(d,aggs)
    data=pd.merge(data,temp,on=d,how='left')

100%|██████████| 10/10 [00:31<00:00,  5.19s/it]


In [79]:
#num_features = ['age', 'balance','day','duration','campaign','pdays','previous']

In [80]:
# 获取数值特征

feature_all_ori = feature1 + ['ID','y']
num_features_add = []
for feature in data.columns.tolist():
    if feature not in feature_all_ori:
        num_features_add.append(feature)
        
num_features += num_features_add
num_features

['age',
 'balance',
 'day',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'job_I_D_count',
 'marital_I_D_count',
 'education_I_D_count',
 'default_I_D_count',
 'housing_I_D_count',
 'loan_I_D_count',
 'contact_I_D_count',
 'month_I_D_count',
 'poutcome_I_D_count',
 'age_I_D_count',
 'balance_I_D_count',
 'day_I_D_count',
 'duration_I_D_count',
 'campaign_I_D_count',
 'pdays_I_D_count',
 'previous_I_D_count',
 'count_day_month_campaign',
 'count_day_month_contact',
 'count_day_month_default',
 'count_day_month_education',
 'count_day_month_housing',
 'count_day_month_job',
 'count_day_month_loan',
 'count_day_month_marital',
 'count_day_month_poutcome',
 'count_day_month_pdays',
 'count_day_month_previous',
 'campaign_contact_count',
 'campaign_contact_nunique',
 'campaign_default_count',
 'campaign_default_nunique',
 'campaign_education_count',
 'campaign_education_nunique',
 'campaign_housing_count',
 'campaign_housing_nunique',
 'campaign_job_count',
 'campaign_job_nunique',
 'ca

In [81]:
# 数据标准化

from sklearn.preprocessing import StandardScaler, MinMaxScaler
mms = StandardScaler()
data[num_features] = mms.fit_transform(data[num_features])

G:\AIAIAI\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
G:\AIAIAI\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [82]:
# 获取独热特征

one_hot_feature =['job', 'marital','education','default','housing','loan','contact','month','poutcome']

In [83]:
# 获取向量化特征

data['new_con'] = data['job'].astype(str)
for i in ['marital', 'education', 'contact','month','poutcome']:
    data['new_con'] = data['new_con'].astype(str) + '_' + data[i].astype(str)
data['new_con'] = data['new_con'].apply(lambda x: ' '.join(x.split('_')))

vector_feature = ['new_con']

In [84]:
for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])

In [85]:
train=data[data.y!=-1]
test=data[data.y==-1]
res=test[['ID']]
train_x=train[num_features]
test_x=test[num_features]

In [86]:
enc = OneHotEncoder()
for feature in one_hot_feature:
    enc.fit(data[feature].values.reshape(-1, 1))
    train_a= enc.transform(train[feature].values.reshape(-1, 1))
    test_a = enc.transform(test[feature].values.reshape(-1, 1))
    train_x= sparse.hstack((train_x, train_a), 'csr')
    test_x = sparse.hstack((test_x, test_a), 'csr')

G:\AIAIAI\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
G:\AIAIAI\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this 

In [87]:
cv=CountVectorizer()
for feature in vector_feature:
    cv.fit(data[feature])
    train_a = cv.transform(train[feature])
    test_a = cv.transform(test[feature])
    train_x = sparse.hstack((train_x, train_a), 'csr')
    test_x = sparse.hstack((test_x, test_a), 'csr')

In [88]:
import lightgbm as lgb
n_splits=5
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
clf = lgb.LGBMClassifier(
    boosting_type="gbdt",
    num_leaves=30,
    reg_alpha=0,
    reg_lambda=0.,
    max_depth=8,
    n_estimators=1500,
    objective='binary',
    metric= 'auc',
    subsample=0.95,
    colsample_bytree=0.7,
    subsample_freq=1,
    learning_rate=0.01,
    random_state=2017
    )

train_y=train['y']
res['pred'] = 0

G:\AIAIAI\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [89]:
for train_idx, val_idx in kfold.split(train_x, train_y):
    clf.random_state = clf.random_state + 1
    train_x1 = train_x[train_idx]
    train_y1 = train_y.loc[train_idx]
    valid_x1 = train_x[val_idx]
    valid_y1 = train_y.loc[val_idx]
    #,(vali_x,vali_y)
    clf.fit(train_x1, train_y1,eval_set=[(train_x1, train_y1),(valid_x1, valid_y1)],eval_metric='auc',early_stopping_rounds=100)
    res['pred'] += clf.predict_proba(test_x)[:,1]

res['pred'] = res['pred']/5
res.to_csv('3_28_4.csv', index=False)

[1]	training's auc: 0.898342	valid_1's auc: 0.887259
Training until validation scores don't improve for 100 rounds.
[2]	training's auc: 0.919391	valid_1's auc: 0.907409
[3]	training's auc: 0.920262	valid_1's auc: 0.908458
[4]	training's auc: 0.932846	valid_1's auc: 0.917406
[5]	training's auc: 0.933696	valid_1's auc: 0.920316
[6]	training's auc: 0.936141	valid_1's auc: 0.923232
[7]	training's auc: 0.937519	valid_1's auc: 0.924613
[8]	training's auc: 0.938904	valid_1's auc: 0.926603
[9]	training's auc: 0.93951	valid_1's auc: 0.928087
[10]	training's auc: 0.940247	valid_1's auc: 0.927115
[11]	training's auc: 0.940917	valid_1's auc: 0.927053
[12]	training's auc: 0.941498	valid_1's auc: 0.927732
[13]	training's auc: 0.941615	valid_1's auc: 0.927416
[14]	training's auc: 0.942193	valid_1's auc: 0.92773
[15]	training's auc: 0.942487	valid_1's auc: 0.928817
[16]	training's auc: 0.942726	valid_1's auc: 0.929091
[17]	training's auc: 0.942704	valid_1's auc: 0.929498
[18]	training's auc: 0.942531	

[300]	training's auc: 0.960404	valid_1's auc: 0.935067
[301]	training's auc: 0.960438	valid_1's auc: 0.935063
[302]	training's auc: 0.960488	valid_1's auc: 0.935072
[303]	training's auc: 0.960533	valid_1's auc: 0.935063
[304]	training's auc: 0.960563	valid_1's auc: 0.93509
[305]	training's auc: 0.96061	valid_1's auc: 0.935111
[306]	training's auc: 0.96064	valid_1's auc: 0.9351
[307]	training's auc: 0.960697	valid_1's auc: 0.935077
[308]	training's auc: 0.960746	valid_1's auc: 0.935115
[309]	training's auc: 0.960786	valid_1's auc: 0.935107
[310]	training's auc: 0.960828	valid_1's auc: 0.935112
[311]	training's auc: 0.960868	valid_1's auc: 0.935125
[312]	training's auc: 0.960914	valid_1's auc: 0.935131
[313]	training's auc: 0.960961	valid_1's auc: 0.935115
[314]	training's auc: 0.960997	valid_1's auc: 0.935098
[315]	training's auc: 0.961037	valid_1's auc: 0.935072
[316]	training's auc: 0.961088	valid_1's auc: 0.935085
[317]	training's auc: 0.961123	valid_1's auc: 0.935085
[318]	training'

[598]	training's auc: 0.971936	valid_1's auc: 0.93602
[599]	training's auc: 0.971975	valid_1's auc: 0.936029
[600]	training's auc: 0.972005	valid_1's auc: 0.936066
[601]	training's auc: 0.97204	valid_1's auc: 0.936053
[602]	training's auc: 0.972077	valid_1's auc: 0.936047
[603]	training's auc: 0.972118	valid_1's auc: 0.936048
[604]	training's auc: 0.972146	valid_1's auc: 0.936045
[605]	training's auc: 0.972174	valid_1's auc: 0.936048
[606]	training's auc: 0.972206	valid_1's auc: 0.936045
[607]	training's auc: 0.972238	valid_1's auc: 0.936037
[608]	training's auc: 0.972261	valid_1's auc: 0.93604
[609]	training's auc: 0.972284	valid_1's auc: 0.93604
[610]	training's auc: 0.972315	valid_1's auc: 0.936049
[611]	training's auc: 0.972347	valid_1's auc: 0.93606
[612]	training's auc: 0.97238	valid_1's auc: 0.93605
[613]	training's auc: 0.972406	valid_1's auc: 0.936056
[614]	training's auc: 0.972449	valid_1's auc: 0.936053
[615]	training's auc: 0.972487	valid_1's auc: 0.936045
[616]	training's 

G:\AIAIAI\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


[1]	training's auc: 0.911231	valid_1's auc: 0.889166
Training until validation scores don't improve for 100 rounds.
[2]	training's auc: 0.92204	valid_1's auc: 0.907462
[3]	training's auc: 0.931879	valid_1's auc: 0.915174
[4]	training's auc: 0.933128	valid_1's auc: 0.916053
[5]	training's auc: 0.933369	valid_1's auc: 0.916519
[6]	training's auc: 0.933607	valid_1's auc: 0.917549
[7]	training's auc: 0.934247	valid_1's auc: 0.917798
[8]	training's auc: 0.935425	valid_1's auc: 0.919034
[9]	training's auc: 0.935747	valid_1's auc: 0.919769
[10]	training's auc: 0.936019	valid_1's auc: 0.919808
[11]	training's auc: 0.936545	valid_1's auc: 0.919795
[12]	training's auc: 0.938776	valid_1's auc: 0.921608
[13]	training's auc: 0.938911	valid_1's auc: 0.921456
[14]	training's auc: 0.939663	valid_1's auc: 0.921391
[15]	training's auc: 0.939492	valid_1's auc: 0.921205
[16]	training's auc: 0.939509	valid_1's auc: 0.92256
[17]	training's auc: 0.940498	valid_1's auc: 0.924243
[18]	training's auc: 0.940543	

...

[508]	training's auc: 0.967782	valid_1's auc: 0.944735
[509]	training's auc: 0.967825	valid_1's auc: 0.944728
[510]	training's auc: 0.967858	valid_1's auc: 0.944699
[511]	training's auc: 0.967889	valid_1's auc: 0.944686
[512]	training's auc: 0.967928	valid_1's auc: 0.944674
[513]	training's auc: 0.967965	valid_1's auc: 0.944688
[514]	training's auc: 0.967996	valid_1's auc: 0.944695
[515]	training's auc: 0.968042	valid_1's auc: 0.944688
[516]	training's auc: 0.968086	valid_1's auc: 0.944678
[517]	training's auc: 0.968123	valid_1's auc: 0.94469
[518]	training's auc: 0.968174	valid_1's auc: 0.944686
[519]	training's auc: 0.968217	valid_1's auc: 0.944725
[520]	training's auc: 0.968271	valid_1's auc: 0.944718
[521]	training's auc: 0.968301	valid_1's auc: 0.944718
[522]	training's auc: 0.968335	valid_1's auc: 0.944717
[523]	training's auc: 0.968366	valid_1's auc: 0.944738
[524]	training's auc: 0.968402	valid_1's auc: 0.944725
[525]	training's auc: 0.968433	valid_1's auc: 0.944719
[526]	train

[806]	training's auc: 0.977197	valid_1's auc: 0.944906
[807]	training's auc: 0.977223	valid_1's auc: 0.944889
[808]	training's auc: 0.977244	valid_1's auc: 0.944875
[809]	training's auc: 0.977273	valid_1's auc: 0.944876
[810]	training's auc: 0.977298	valid_1's auc: 0.944896
[811]	training's auc: 0.977326	valid_1's auc: 0.944886
[812]	training's auc: 0.977354	valid_1's auc: 0.944899
[813]	training's auc: 0.977387	valid_1's auc: 0.944886
[814]	training's auc: 0.977412	valid_1's auc: 0.944888
[815]	training's auc: 0.977431	valid_1's auc: 0.944887
[816]	training's auc: 0.977453	valid_1's auc: 0.944867
[817]	training's auc: 0.977474	valid_1's auc: 0.944883
[818]	training's auc: 0.977511	valid_1's auc: 0.944894
[819]	training's auc: 0.977538	valid_1's auc: 0.944897
[820]	training's auc: 0.977548	valid_1's auc: 0.944895
[821]	training's auc: 0.977573	valid_1's auc: 0.944896
[822]	training's auc: 0.97761	valid_1's auc: 0.944896
[823]	training's auc: 0.977635	valid_1's auc: 0.944897
[824]	train

[254]	training's auc: 0.956756	valid_1's auc: 0.941742
[255]	training's auc: 0.956815	valid_1's auc: 0.941769
[256]	training's auc: 0.956866	valid_1's auc: 0.941787
[257]	training's auc: 0.956941	valid_1's auc: 0.941748
[258]	training's auc: 0.956976	valid_1's auc: 0.941771
[259]	training's auc: 0.957037	valid_1's auc: 0.941742
[260]	training's auc: 0.957112	valid_1's auc: 0.94174
[261]	training's auc: 0.957148	valid_1's auc: 0.941748
[262]	training's auc: 0.957208	valid_1's auc: 0.941762
[263]	training's auc: 0.957251	valid_1's auc: 0.941807
[264]	training's auc: 0.957326	valid_1's auc: 0.941843
[265]	training's auc: 0.957372	valid_1's auc: 0.941852
[266]	training's auc: 0.957414	valid_1's auc: 0.941854
[267]	training's auc: 0.957479	valid_1's auc: 0.941882
[268]	training's auc: 0.957537	valid_1's auc: 0.941883
[269]	training's auc: 0.957565	valid_1's auc: 0.941873
[270]	training's auc: 0.957609	valid_1's auc: 0.941901
[271]	training's auc: 0.95766	valid_1's auc: 0.941895
[272]	traini

[552]	training's auc: 0.969399	valid_1's auc: 0.943257
[553]	training's auc: 0.969446	valid_1's auc: 0.943279
[554]	training's auc: 0.969476	valid_1's auc: 0.943272
[555]	training's auc: 0.969516	valid_1's auc: 0.94327
[556]	training's auc: 0.969556	valid_1's auc: 0.943281
[557]	training's auc: 0.969597	valid_1's auc: 0.943293
[558]	training's auc: 0.969627	valid_1's auc: 0.943302
[559]	training's auc: 0.969659	valid_1's auc: 0.943297
[560]	training's auc: 0.969687	valid_1's auc: 0.94329
[561]	training's auc: 0.969713	valid_1's auc: 0.943289
[562]	training's auc: 0.969759	valid_1's auc: 0.943283
[563]	training's auc: 0.969795	valid_1's auc: 0.943277
[564]	training's auc: 0.969832	valid_1's auc: 0.94329
[565]	training's auc: 0.96986	valid_1's auc: 0.943275
[566]	training's auc: 0.969906	valid_1's auc: 0.943301
[567]	training's auc: 0.969946	valid_1's auc: 0.943285
[568]	training's auc: 0.969988	valid_1's auc: 0.94327
[569]	training's auc: 0.970034	valid_1's auc: 0.943298
[570]	training'

G:\AIAIAI\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
